Project 🚧

Walmart's marketing service has asked you to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made.

Goals 🎯

The project can be divided into three steps:

Part 1 : make an EDA and all the necessary preprocessings to prepare data for machine learning
Part 2 : train a linear regression model (baseline)
Part 3 : avoid overfitting by training a regularized regression model

Deliverable 📬

To complete this project, your team should:

1) Create some visualizations
2) Train at least one linear regression model on the dataset, that predicts the amount of weekly sales as a function of the other variables
3) Assess the performances of the model by using a metric that is relevant for regression problems
4) Interpret the coefficients of the model to identify what features are important for the prediction
5) Train at least one model with regularization (Lasso or Ridge) to reduce overfitting
Explanatory variables (X) We need to identify which columns contain categorical variables and which columns contain numerical variables, as they will be treated differently.

Categorical variables : Store, Holiday_Flag
Numerical variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, DayOfWeek

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import plotly.io as pio


pio.templates.default = "plotly_dark"
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('Walmart_Store_sales.csv')

In [3]:
# Function to find missing values
def missing_values(df, norows):  # input by the df and the number of rows that you want to show
    total = df.isnull().sum().sort_values(ascending=False)
    percent = ((df.isnull().sum().sort_values(ascending=False) / df.shape[0]) * 100).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data.head(norows)

In [4]:
from IPython.core.display_functions import display

# Basic stats
print("Number of rows : {}".format(df.shape[0]))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [5]:
print("Missing values: ")
missing_values(df, 20)

Missing values: 


,Total,Percent
Date,18,12.000000
Temperature,18,12.000000
Unemployment,15,10.000000
Weekly_Sales,14,9.333333
Fuel_Price,14,9.333333
Holiday_Flag,12,8.000000
CPI,12,8.000000
Store,0,0.000000


In [6]:
# Splitting the date column into year, month, day and day of week as type float64
df['Date_dt'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Year'] = df['Date_dt'].dt.year
df['Month'] = df['Date_dt'].dt.month
df['Day'] = df['Date_dt'].dt.day
df['DayOfWeek'] = df['Date_dt'].dt.dayofweek
df['WeekOfYear'] = df['Date_dt'].dt.weekofyear

df['Store'] = df['Store'].astype(str)
df['Holiday_Flag'] = df['Holiday_Flag'].astype(str).str.replace('.0', '')

/var/folders/5q/4k55d_7j1kq3rt9wy30qwsk40000gn/T/ipykernel_5864/2586018816.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['WeekOfYear'] = df['Date_dt'].dt.weekofyear
/var/folders/5q/4k55d_7j1kq3rt9wy30qwsk40000gn/T/ipykernel_5864/2586018816.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Holiday_Flag'] = df['Holiday_Flag'].astype(str).str.replace('.0', '')


In [7]:
df.dtypes

Store                   object
Date                    object
Weekly_Sales           float64
Holiday_Flag            object
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
Date_dt         datetime64[ns]
Year                   float64
Month                  float64
Day                    float64
DayOfWeek              float64
WeekOfYear             float64
dtype: object

In [8]:
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Date_dt,Year,Month,Day,DayOfWeek,WeekOfYear
0,6.0,18-02-2011,1572117.54,nan,59.61,3.045,214.777523,6.858,2011-02-18,2011.0,2.0,18.0,4.0,7.0
1,13.0,25-03-2011,1807545.43,0,42.38,3.435,128.616064,7.470,2011-03-25,2011.0,3.0,25.0,4.0,12.0
2,17.0,27-07-2012,NaN,0,NaN,NaN,130.719581,5.936,2012-07-27,2012.0,7.0,27.0,4.0,30.0
3,11.0,NaN,1244390.03,0,84.57,NaN,214.556497,7.346,NaT,NaN,NaN,NaN,NaN,NaN
4,6.0,28-05-2010,1644470.66,0,78.89,2.759,212.412888,7.092,2010-05-28,2010.0,5.0,28.0,4.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,14.0,18-06-2010,2248645.59,0,72.62,2.780,182.442420,8.899,2010-06-18,2010.0,6.0,18.0,4.0,24.0
146,7.0,NaN,716388.81,nan,20.74,2.778,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
147,17.0,11-06-2010,845252.21,0,57.14,2.841,126.111903,NaN,2010-06-11,2010.0,6.0,11.0,4.0,23.0
148,8.0,12-08-2011,856796.10,0,86.05,3.638,219.007525,NaN,2011-08-12,2011.0,8.0,12.0,4.0,32.0


In [9]:
df["DayOfWeek"].value_counts()

4.0    132
Name: DayOfWeek, dtype: int64

In [10]:
df["Unemployment"].isna().sum()

15

In [11]:
df.drop(['DayOfWeek'], axis=1, inplace=True)
df.drop(['Date'], axis=1, inplace=True)
df.drop(['Date_dt'], axis=1, inplace=True)
df.dropna(subset=['Weekly_Sales'], inplace=True)
df.shape

(136, 11)

In [12]:
fig = px.bar(df, x="Store", y="Weekly_Sales")
fig.show()

In [13]:
features_list = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

# make a grid of boxplot of outliers for each feature in features_list
fig = make_subplots(1, 4)
for feature in features_list:
    fig.add_trace(go.Box(y=df[feature], name=feature, boxpoints="all"), row=1, col=features_list.index(
        feature) + 1)  # add a boxplot for each feature, +1 added for column index to start at 1
fig.update_layout(height=600, width=800, title_text="Boxplots of outliers")
fig.show()

In [14]:
# We can see from previous boxplots that there are outliers only in the Unemployment feature
# Verifying this hypothesis
for feature in features_list:
    upper_bound = df[feature].mean() + 3 * df[feature].std()
    lower_bound = df[feature].mean() - 3 * df[feature].std()
    # print the number of outliers for each feature
    print(
        f"Number of outliers for {feature} : {df[(df[feature] > upper_bound) | (df[feature] < lower_bound)].shape[0]}")


Number of outliers for Temperature : 0
Number of outliers for Fuel_Price : 0
Number of outliers for CPI : 0
Number of outliers for Unemployment : 5


In [15]:
# Removing outliers for the Unemployment feature
upper_bound = df['Unemployment'].mean() + 3 * df['Unemployment'].std()
print(upper_bound)
lower_bound = df['Unemployment'].mean() - 3 * df['Unemployment'].std()
print(lower_bound)
# drop outliers but keep the NaN values
# outliers = df[(df['Unemployment'] < upper_bound) & (df['Unemployment'] > lower_bound)]


df = df[(df['Unemployment'] < upper_bound) & (df['Unemployment'] > lower_bound) | (df['Unemployment'].isna())]
df.shape


12.523867092274017
2.807296842152213


(131, 11)

In [16]:
df_pivot = df.pivot_table(index=['Store'], values='Weekly_Sales', aggfunc=['count', 'sum'], margins=True,
                          margins_name='Total')
df_pivot

,count,sum
,Weekly_Sales,Weekly_Sales
Store,,
1.0,9,1.406257e+07
10.0,5,9.110529e+06
11.0,3,4.571240e+06
13.0,9,1.797512e+07
14.0,9,1.883591e+07
15.0,4,2.534322e+06
16.0,4,2.061271e+06
17.0,7,6.241242e+06


In [17]:
df_pivot = df.pivot_table(index=['Year'], values='Weekly_Sales', aggfunc=np.sum, margins=True, margins_name='Total')
df_pivot

,Weekly_Sales
Year,
2010.0,6.217854e+07
2011.0,4.544811e+07
2012.0,3.559122e+07
Total,1.432179e+08


In [18]:
target_name = 'Weekly_Sales'
categorical_features = ['Store', 'Holiday_Flag']
numeric_features = df.columns.drop(categorical_features + [target_name])
print(f"Target Y: {target_name}")
print(f"Categorical features: {categorical_features}")
print(f"Numeric features: {numeric_features}")

Target Y: Weekly_Sales
Categorical features: ['Store', 'Holiday_Flag']
Numeric features: Index(['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month',
       'Day', 'WeekOfYear'],
      dtype='object')


In [19]:
Y = df.loc[:, target_name]
X = df.drop(target_name, axis=1)
# print(Y.head())
# print(X.head())

In [20]:
from sklearn.compose import ColumnTransformer

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'),),  # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),  # missing values will be replaced by most frequent value
        ('encoder', OneHotEncoder(drop='first'))
        # first column will be dropped to avoid creating correlations between features
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocessings on train set
X_train = preprocessor.fit_transform(X_train)
# Preprocessings on test set
X_test = preprocessor.transform(X_test)

In [21]:
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression()

In [22]:
scores = cross_val_score(regressor, X_train, Y_train, cv=10)
print(max(scores))
print(min(scores))
print(scores.mean())
print(scores.std())
print(scores)

0.9929250438980405
0.8978181490446419
0.938120535517079
0.0319542427040292
[0.90037937 0.90354231 0.92572677 0.89781815 0.96471214 0.99292504
 0.9197666  0.9642483  0.94120806 0.97087862]


In [23]:
print('R2 on train: ', regressor.score(X_train, Y_train))
print('R2 on test: ', regressor.score(X_test, Y_test))

df_scores = pd.DataFrame(columns=['model', 'R2 score on training set', 'R2 score on test set'])
df_scores = df_scores.append({'model': 'Linear Regression', 'R2 score on training set': regressor.score(X_train, Y_train),
                              'R2 score on test set': regressor.score(X_test, Y_test)}, ignore_index=True)
print(df_scores)

R2 on train:  0.9746583650396601
R2 on test:  0.938808426865494
               model  R2 score on training set  R2 score on test set
0  Linear Regression                  0.974658              0.938808


/var/folders/5q/4k55d_7j1kq3rt9wy30qwsk40000gn/T/ipykernel_5864/2773733891.py:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [24]:
regressor.coef_

array([  -38567.48408714,   -43705.89834529,    97354.79760579,
         -93422.42574561,    -8170.20389973,   887165.71547804,
          28140.729382  ,  -819528.05278886,   702109.59236386,
           3663.27671962,   557765.12698622,   711941.66581594,
        -645459.56031739, -1116971.26662946,  -620916.39836111,
        -154497.29748515,   108932.76519185,   386673.51672595,
         366888.98138992, -1280759.09960742,   663589.09984391,
       -1405352.30302366,   -28504.86155331,  -902563.11757531,
        -765863.75655361, -1284017.3554019 ,   -47672.95335937,
         -12633.89450271])

In [25]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_:  # loop over pipelines
    if name == 'num':  # if pipeline is for numeric variables
        features = features_list  # just get the names of columns to which it has been applied
    else:  # if pipeline is for categorical variables
        features = pipeline.named_steps[
            'encoder'].get_feature_names_out()  # get output columns names from OneHotEncoder
    column_names.extend(features)  # concatenate features names

print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'WeekOfYear', 'x0_10.0', 'x0_11.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_2.0', 'x0_20.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x1_1', 'x1_nan']


In [26]:
coefs = pd.DataFrame(index=column_names, data=regressor.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Temperature,-3.856748e+04
Fuel_Price,-4.370590e+04
CPI,9.735480e+04
Unemployment,-9.342243e+04
Year,-8.170204e+03
Month,8.871657e+05
Day,2.814073e+04
WeekOfYear,-8.195281e+05
x0_10.0,7.021096e+05
x0_11.0,3.663277e+03


In [27]:
feature_importance = abs(coefs).sort_values(by='coefficients')
feature_importance

,coefficients
x0_11.0,3.663277e+03
Year,8.170204e+03
x1_nan,1.263389e+04
Day,2.814073e+04
x0_6.0,2.850486e+04
Temperature,3.856748e+04
Fuel_Price,4.370590e+04
x1_1,4.767295e+04
Unemployment,9.342243e+04
CPI,9.735480e+04


In [28]:
# Plot coefficients
fig = px.bar(feature_importance, orientation='h')
fig.update_layout(showlegend=False,
                  margin={'l': 150}  # to avoid cropping of column names
                  )
fig.show()

In [29]:
regressor = Ridge()
# scores = cross_val_score(regressor, X_train, Y_train, cv=3)
# print('The cross-validated R2-score is : ', scores.mean())
# print('The standard deviation is : ', scores.std())

In [30]:
# Perform grid search
print("Grid search...")
regressor = Ridge(max_iter=100000)
# Grid of values to be tested
params = {
    "alpha": [0.001*i for i in range(1, 101)]
}
gridsearch = GridSearchCV(regressor, param_grid=params, cv=10)  # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

# PERFORMANCE ASSESSMENT

# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

df_scores = df_scores.append(
    {'model': 'Ridge Regression', 'R2 score on training set': gridsearch.score(X_train, Y_train),
     'R2 score on test set': gridsearch.score(X_test, Y_test)}, ignore_index=True)
df_scores.head()

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.026000000000000002}
Best R2 score :  0.9408044263049264
R2 score on training set :  0.9744642367338602
R2 score on test set :  0.9434416679459804


/var/folders/5q/4k55d_7j1kq3rt9wy30qwsk40000gn/T/ipykernel_5864/3393956483.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,model,R2 score on training set,R2 score on test set
0,Linear Regression,0.974658,0.938808
1,Ridge Regression,0.974464,0.943442


In [31]:
# Perform grid search
print("Grid search...")
regressor = Lasso(max_iter=100000)
# Grid of values to be tested
params = {
    'alpha': [1*i for i in range(1, 1000)]
}
gridsearch = GridSearchCV(regressor, param_grid=params, cv=10)  # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

# PERFORMANCE ASSESSMENT

# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("R2 score on test set : ", gridsearch.score(X_test, Y_test))

df_scores = df_scores.append(
    {'model': 'Lasso Regression', 'R2 score on training set': gridsearch.score(X_train, Y_train),
     'R2 score on test set': gridsearch.score(X_test, Y_test)}, ignore_index=True)
df_scores

Grid search...


/Users/pryda/.pyenv/versions/3.11.0/envs/bootcamp/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.355e+10, tolerance: 3.857e+09

/Users/pryda/.pyenv/versions/3.11.0/envs/bootcamp/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.346e+11, tolerance: 3.920e+09

/Users/pryda/.pyenv/versions/3.11.0/envs/bootcamp/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

...Done.
Best hyperparameters :  {'alpha': 661}
Best R2 score :  0.943369496376633
R2 score on training set :  0.9737064397269549
R2 score on test set :  0.9487871569696432


/var/folders/5q/4k55d_7j1kq3rt9wy30qwsk40000gn/T/ipykernel_5864/340122962.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,model,R2 score on training set,R2 score on test set
0,Linear Regression,0.974658,0.938808
1,Ridge Regression,0.974464,0.943442
2,Lasso Regression,0.973706,0.948787
